In [1]:
import pandas as pd
import numpy as np
import json
import feather
from operator import itemgetter

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [204]:
ml_papers = pd.read_pickle("dblp.pkl")

In [137]:
citation = ml_papers[["n_citation"]]
citation.to_feather('citation_only.feather')

,authors,id,n_citation,references,title,venue,year
0,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998
5,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",005ce28f-ed77-4e97-afdc-a296137186a1,0,NaN,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,international conference on computer graphics ...,2009
6,"[Jovan Dj. Golic, Guglielmo Morgari]",00638a94-23bf-4fa6-b5ce-40d799c65da7,2,NaN,Vectorial fast correlation attacks.,,2004
7,"[Guzin Ulutas, Mustafa Ulutas, Vasif V. Nabiyev]",00701b05-684f-45f9-b281-425abfec482c,0,"[5626736c-e434-4e2d-8405-54940fab88ab, 8e87e87...",Improved Secret Image Sharing Method By Encodi...,international symposium on computer and inform...,2011
8,"[Pranay Chaudhuri, Hussein Thompson]",00745041-3636-4d18-bbec-783c4278c40d,0,NaN,A Self-Stabilizing Algorithm for Finding the C...,parallel and distributed processing techniques...,2003
9,"[Dominik Szajerman, Adam Jurczyński]",00964544-cbe2-4da9-bb5a-03333160eb34,0,"[3fcd7cdc-20e6-4ea3-a41c-db126fcc5cfe, bf3a11c...",Fur Visualisation for Computer Game Engines an...,international conference on computer vision an...,2014


In [205]:
top_10_venues = list(ml_papers[["venue", "n_citation"]]
                     .groupby(["venue"])
                     .sum()
                     .reset_index()
                     .sort_values(by = "n_citation", ascending = False)["venue"][1:11]
                    )
top_10_venues

['IEEE Transactions on Pattern Analysis and Machine Intelligence',
 'IEEE Transactions on Information Theory',
 'Lecture Notes in Computer Science',
 'international conference on robotics and automation',
 'Bioinformatics',
 'computer vision and pattern recognition',
 'Communications of The ACM',
 'Nucleic Acids Research',
 'international conference on acoustics, speech, and signal processing',
 'international conference on computer graphics and interactive techniques']

In [206]:
ml_papers = ml_papers[ml_papers.venue.isin(top_10_venues)]

In [207]:
ml_papers.head()

,authors,id,n_citation,references,title,venue,year
287,"[N. Sai Shankar, Sunghyun Choi]",11f0bd37-ae5a-43e6-b14a-a59bc00fdd90,50,NaN,QoS Signaling for Parameterized Traffic in IEE...,Lecture Notes in Computer Science,2002
331,"[C. Graciani Díaz, Francisco-Jesús Martín-Mate...",155dec16-36d6-44f4-976b-1afb5d1924af,50,[c17481ca-9511-4793-8dad-a2486e0b2713],Specification of Adleman's Restricted Model Us...,Lecture Notes in Computer Science,2002
484,"[Van-Nam Huynh, Yoshiteru Nakamori, Tu Bao Ho]",20f0a5af-395a-487f-854d-9888295f2508,4,"[5fe2f07f-4093-4c1a-94e6-39666362ec48, b2e6bac...",Utility function induced by fuzzy target in pr...,Lecture Notes in Computer Science,2006
489,"[Yisheng Guan, Teresa R. Ho, Hong Zhang]",210d708b-080c-46ce-850f-0588450de08b,0,NaN,An Integrated Robotic Hand/Simulator System fo...,international conference on robotics and autom...,2001
544,"[Makoto Matsushita, Makoto Oshita, Hajimu Iida...",25e09f2e-c606-469f-a763-5d20b4704a86,7,"[269663ea-4dc4-4e99-86da-b04240bfdf5c, 3ff297f...",Distributed Process Management System Based on...,Lecture Notes in Computer Science,1998


## LDA

In [156]:
documents = ml_papers[['id', 'title']]

In [157]:
documents.head()

,id,title
287,11f0bd37-ae5a-43e6-b14a-a59bc00fdd90,QoS Signaling for Parameterized Traffic in IEE...
331,155dec16-36d6-44f4-976b-1afb5d1924af,Specification of Adleman's Restricted Model Us...
484,20f0a5af-395a-487f-854d-9888295f2508,Utility function induced by fuzzy target in pr...
489,210d708b-080c-46ce-850f-0588450de08b,An Integrated Robotic Hand/Simulator System fo...
544,25e09f2e-c606-469f-a763-5d20b4704a86,Distributed Process Management System Based on...


In [158]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(3)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tniyomkarn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [159]:
documents['index'] = documents.index

/Users/tniyomkarn/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [160]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [161]:
processed_docs = documents['title'].map(preprocess)
processed_docs[:10]

287     [signal, parameter, traffic, ieee, wireless, lan]
331     [specif, adleman, restrict, model, autom, reas...
484     [util, function, induc, fuzzi, target, probabi...
489     [integr, robot, hand, simul, tele, manipul, in...
544     [distribut, process, manag, base, object, cent...
1096                      [seiz, power, shader, storytel]
1135             [progress, mobil, microfabr, millirobot]
1307              [critic, comput, academ, inevit, divis]
1353    [holon, multi, agent, robust, flexibl, reliabl...
1431    [support, student, draw, diagram, assess, comp...
Name: title, dtype: object

In [162]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 ieee
1 lan
2 parameter
3 signal
4 traffic
5 wireless
6 adleman
7 autom
8 experi
9 lipton
10 model


In [163]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [164]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(248, 1), (694, 1), (1024, 1)]

In [165]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 248 ("maximum") appears 1 time.
Word 694 ("valu") appears 1 time.
Word 1024 ("mean") appears 1 time.


In [166]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4616494462947011),
 (1, 0.5472322759358625),
 (2, 0.4435473513780698),
 (3, 0.241459641910254),
 (4, 0.3797916281586823),
 (5, 0.2968803336438776)]


In [180]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [181]:
for idx, topic in lda_model.print_topics(-1, 20):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"estim" + 0.020*"base" + 0.019*"imag" + 0.017*"model" + 0.013*"detect" + 0.011*"structur" + 0.010*"algorithm" + 0.009*"method" + 0.008*"recognit" + 0.008*"sequenc" + 0.007*"signal" + 0.007*"data" + 0.007*"spars" + 0.007*"match" + 0.007*"robust" + 0.007*"align" + 0.007*"reconstruct" + 0.007*"dimension" + 0.007*"adapt" + 0.006*"filter"
Topic: 1 
Words: 0.022*"data" + 0.018*"analysi" + 0.017*"databas" + 0.016*"model" + 0.016*"base" + 0.012*"gene" + 0.011*"genom" + 0.011*"protein" + 0.011*"interact" + 0.010*"tool" + 0.010*"design" + 0.009*"integr" + 0.009*"comput" + 0.009*"languag" + 0.008*"process" + 0.008*"express" + 0.007*"structur" + 0.007*"learn" + 0.007*"softwar" + 0.007*"studi"
Topic: 2 
Words: 0.038*"robot" + 0.032*"control" + 0.017*"system" + 0.016*"inform" + 0.013*"base" + 0.013*"network" + 0.012*"manipul" + 0.012*"dynam" + 0.012*"channel" + 0.010*"optim" + 0.010*"plan" + 0.010*"design" + 0.009*"motion" + 0.008*"multi" + 0.007*"communic" + 0.007*"adapt" + 0

In [193]:
topic_dict = {
    0: "Pattern Recognition and Machine Learning",
    1: "Bioinformatics",
    2: "Robotics and Automation",
    3: "Computer Vision",
    4: "Signal and Speech Processing",
}

In [194]:
topic_prob = lda_model.get_document_topics(bow_corpus, per_word_topics=True)

In [195]:
# for doc_topics, word_topics, phi_values in topics:
#     print('New Document \n')
#     print ('Document topics:', doc_topics)
# #     print 'Word topics:', word_topics
# #     print 'Phi values:', phi_values
#     print(" ")
#     print('-------------- \n')

In [196]:
topics = [max(x[0],key=itemgetter(1))[0]  for x in topic_prob]

In [197]:
topics = np.array(topics)

## Convert authors and references

In [208]:
ml_papers['topic'] = topics

In [209]:
ml_papers.authors = ml_papers.authors.apply(', '.join)
ml_papers.loc[ml_papers['references'].isnull(),['references']] = ml_papers.loc[ml_papers['references'].isnull(),'references'].apply(lambda x: [])
ml_papers.references = ml_papers.references.apply(', '.join)

In [210]:
ml_papers = ml_papers.replace({"topic": topic_dict})

In [211]:
ml_papers.head()

,authors,id,n_citation,references,title,venue,year,topic
287,"N. Sai Shankar, Sunghyun Choi",11f0bd37-ae5a-43e6-b14a-a59bc00fdd90,50,,QoS Signaling for Parameterized Traffic in IEE...,Lecture Notes in Computer Science,2002,Computer Vision
331,"C. Graciani Díaz, Francisco-Jesús Martín-Mateo...",155dec16-36d6-44f4-976b-1afb5d1924af,50,c17481ca-9511-4793-8dad-a2486e0b2713,Specification of Adleman's Restricted Model Us...,Lecture Notes in Computer Science,2002,Bioinformatics
484,"Van-Nam Huynh, Yoshiteru Nakamori, Tu Bao Ho",20f0a5af-395a-487f-854d-9888295f2508,4,"5fe2f07f-4093-4c1a-94e6-39666362ec48, b2e6bac3...",Utility function induced by fuzzy target in pr...,Lecture Notes in Computer Science,2006,Signal and Speech Processing
489,"Yisheng Guan, Teresa R. Ho, Hong Zhang",210d708b-080c-46ce-850f-0588450de08b,0,,An Integrated Robotic Hand/Simulator System fo...,international conference on robotics and autom...,2001,Robotics and Automation
544,"Makoto Matsushita, Makoto Oshita, Hajimu Iida,...",25e09f2e-c606-469f-a763-5d20b4704a86,7,"269663ea-4dc4-4e99-86da-b04240bfdf5c, 3ff297f0...",Distributed Process Management System Based on...,Lecture Notes in Computer Science,1998,Bioinformatics


In [212]:
len(ml_papers)

137608

In [213]:
ml_papers.reset_index(drop=True).to_feather('ml_papers_2.feather')